In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use('seaborn-colorblind')

# 3x3 Prediction vs Reference Scatterplot
With the model_dir variable it is possible to choose between the
different neural network architectures

In [ ]:
# Model directory / Network architecture (e.g. DataAugmented, OriginalFukami,...)
model_dir = "Subpixel"

# Define the range of the time series
var = {0: "Hs", 1: "Tm02", 2: "Dir"}

# Define the points where the time series are extracted (in pixel)
xi = [69, 86, 69]
yi = [94, 104, 77] 

# Computer number of extraction points for easier acess
npts = len(xi)

# Save the figure?
save = False

In [ ]:
# Constants to easily change font sizes
LABELSIZE = 15
TICKLABELSIZE = 13
LEGENDFONTSIZE = 12
TITLESIZE = 15
TEXTSIZE = 13

# Various plot labels
xlabel = {0: r"$H_S$ ", 1: r"$T_{m02}$ ", 2: r"Dir "}
units = {0: "[m]", 1: "[s]", 2: "[°]"}

label1 = "Model Prediction"
label2 = "LR Input"
label3 = "HR Ground Truth"
labels = [label1, label2, label3]

fig, axs = plt.subplots(3, 3, figsize=(14,14))

for nvar in range(len(var)):
    # Determine max- and minval for each column to determine axis limits
    min_val = np.infty
    max_val = -np.infty
    for npt in range(npts):
        # Load time series
        path_dir = "./Data/Time_Series/{}".format(model_dir)
        
        path_arr = path_dir + "/Var_{}_posxy_{}_{}.npy"
        path_arr = path_arr.format(var[nvar], xi[npt], yi[npt])

        arr = np.load(path_arr)

        prediction = arr[0,:]
        HR = arr[1,:]
        LR = arr[2,:]
        
        # Avoid problems with NaN while fitting
        if nvar == 2:
            LR = LR[~np.isnan(LR)]
            HR = HR[~np.isnan(HR)]
            prediction = prediction[~np.isnan(prediction)]
            
        m, _ = np.polyfit(HR, prediction, 1)
        
        # Create a text and textbox to display the slope
        text = "m={:.3f}".format(m)
        
        textbox = axs[npt,nvar].annotate(text, (0.02, 0.95),
                                         xycoords="axes fraction",
                                         fontsize=TEXTSIZE,
                                         bbox=dict(boxstyle="round",
                                                   facecolor='none',
                                                   edgecolor="grey"))
        
        # Draw the scatter plots
        line1 = axs[npt,nvar].scatter(HR, prediction, s=1, label=label1, zorder=2)
        line2 = axs[npt,nvar].scatter(HR, LR, s=1, label=label2, zorder=1)
        line3, = axs[npt,nvar].plot(HR, HR, '-k', linewidth=0.5, label=label3)
        
        axs[0,nvar].set_title(xlabel[nvar] + units[nvar], fontsize=TITLESIZE)
        axs[npt,nvar].tick_params(axis='both', labelsize=TICKLABELSIZE)
        
        axs[1,0].set_ylabel("Low-Resolution Inputs and Neural Network predictions at P1-P3", size=LABELSIZE)
        axs[2,1].set_xlabel("High-Resolution SWAN Results", size=LABELSIZE)
        
        # Determine x-, and y-limits for each column
        max_val_cur = max(np.nanmax(LR), np.nanmax(prediction), np.nanmax(HR))
        min_val_cur = min(np.nanmin(LR), np.nanmin(prediction), np.nanmin(HR))
        
        if max_val < max_val_cur:
            max_val = max_val_cur
        if min_val > min_val_cur:
            min_val = min_val_cur        
        
        
    axs[0, nvar].set_xlim(min_val, max_val)
    axs[0, nvar].set_ylim(min_val, max_val)

    axs[1, nvar].set_xlim(min_val, max_val)
    axs[1, nvar].set_ylim(min_val, max_val)

    axs[2, nvar].set_xlim(min_val, max_val)
    axs[2, nvar].set_ylim(min_val, max_val)
        

handles = [line1, line2, line3]
fig.legend(handles, labels, loc=[0.3, 0.003], markerscale=8,
           fancybox=True, shadow=True, ncol=3, fontsize=LEGENDFONTSIZE)

plt.tight_layout()

fig.subplots_adjust(bottom=0.073)


# Output folder and file name
fdir = "Plots"
fname = "ReferenceVsPrediction_mdir{}.png".format(model_dir)

if save:
    if not os.path.isdir(fdir):
        os.makedirs(fdir)
    plt.savefig(os.path.join(fdir, fname))